In [ ]:
from enum import Enum
import copy
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
class Status(Enum):
    """
    Enum representing possible statuses for a cell
    """

    UNOCCUPIED = 0
    CLUELESS = 1
    GOSSIP_SPREADER = 2
    SECRET_KEEPER = 3

In [ ]:
class Cell:
    """
    Represents a cell in a grid, with a status and spreading probability.

    Attributes:
        status (Status): The current status of the cell (e.g. UNOCCUPIED, CLUELESS)
        spreading_prob (float): The probability of becoming a gossip spreader, between 0 and 1

    Methods:
        get_status(): Returns the current status of the cell
        set_status(status): Sets the status of the cell to the provided value
        is_gossip_spreader(): Returns True if the cell's status is 'GOSSIP_SPREADER', False otherwise
        is_clueless(): Returns True if the cell's status is 'CLUELESS', False otherwise
        get_spreading_prob(): Returns the current spreading probability of the cell
        set_spreading_prob(spreading_prob): Sets the spreading probability of the cell
    """

    def __init__(self, status, spreading_prob=0):
        """
        Initializes a new Cell instance with a status and spreading probability.

        Parameters:
            status (Status): The initial status of the cell. It must be a valid status from the Status enum.
            spreading_prob (float, optional): The probability of the cell becoming a gossip spreader. Defaults to 0.

        Raises:
            ValueError: If the status is not an instance of the Status enum.
        """
        if not isinstance(status, Status):
            raise ValueError(
                f"status must be an instance of Status enum, got {type(status)}"
            )

        self.status = status
        self.spreading_prob = spreading_prob

    def get_status(self):
        """
        Returns the current status of the cell.

        Returns:
            Status: The current status of the cell.
        """
        return self.status

    def set_status(self, status):
        """
        Sets the status of the cell.

        Parameters:
            status (Status): The new status to set for the cell. It must be a valid status from the Status enum.

        Raises:
            ValueError: If the provided status is not an instance of the Status enum.
        """
        if not isinstance(status, Status):
            raise ValueError(
                f"status must be an instance of Status enum, got {type(status)}"
            )

        self.status = status

    def is_gossip_spreader(self):
        """
        Checks if the cell is a gossip spreader.

        Returns:
            bool: True if the cell's status is 'GOSSIP_SPREADER', False otherwise.
        """
        return self.status == Status.GOSSIP_SPREADER

    def is_clueless(self):
        """
        Checks if the cell is clueless.

        Returns:
            bool: True if the cell's status is 'CLUELESS', False otherwise.
        """
        return self.status == Status.CLUELESS

    def get_spreading_prob(self):
        """
        Returns the current spreading probability of the cell.

        Returns:
            float: The current spreading probability.
        """
        return self.spreading_prob

    def set_spreading_prob(self, spreading_prob):
        """
        Sets the spreading probability of the cell, ensuring it is between 0 and 1.

        Parameters:
            spreading_prob (float): The new spreading probability to set for the cell.

        Raises:
            ValueError: If spreading_prob is not between 0 and 1 (inclusive).
        """
        if not (0 <= spreading_prob <= 1):
            raise ValueError(
                f"spreading_prob must be between 0 and 1, got {spreading_prob}"
            )

        self.spreading_prob = spreading_prob

In [ ]:
class Grid:
    def __init__(self, size, d, spread_threshold):
        self.size = size
        self.lecture_hall = []
        self.d = d
        self.spread_threshold = spread_threshold

    def initialize_board(self):
        self.lecture_hall = [
            [Cell(Status.UNOCCUPIED) for _ in range(self.size)]
            for _ in range(self.size)
        ]

        # Total number of cells
        total_cells = self.size * self.size

        # Number of cells to set to 1
        cells_to_fill = int(total_cells * self.d)

        # Flatten grid into a list of coordinates
        all_cells = [(i, j) for i in range(self.size) for j in range(self.size)]

        # Randomly choose `cells_to_fill` coordinates
        selected_cells = random.sample(all_cells, cells_to_fill)

        # Set selected cells to 1
        for i, j in selected_cells:
            self.lecture_hall[i][j].set_status(Status.CLUELESS)
            self.lecture_hall[i][j].set_spreading_prob(np.random.uniform(0, 1))

    def set_spreader(self, i, j):
        self.lecture_hall[i][j].set_status(Status.GOSSIP_SPREADER)

    def get_neighbours(self, i, j):
        neighbours = []

        # Top neighbour
        if i > 0:
            neighbours.append(self.lecture_hall[i - 1][j])

        # Bottom neighbours
        if i < self.size - 1:
            neighbours.append(self.lecture_hall[i + 1][j])

        # Left neighbours
        if j > 0:
            neighbours.append(self.lecture_hall[i][j - 1])

        # Right neighbours
        if j < self.size - 1:
            neighbours.append(self.lecture_hall[i][j + 1])

        return neighbours

    def update_grid(self):
        """
        Run one time step
        """

        new_lecture_hall = copy.deepcopy(self.lecture_hall)

        for i in range(self.size):
            for j in range(self.size):
                current_cell = self.lecture_hall[i][j]

                if not current_cell.is_clueless():
                    continue

                neighbours = self.get_neighbours(i, j)

                neighbour_statuses = [
                    neighbour.is_gossip_spreader() for neighbour in neighbours
                ]

                if True in neighbour_statuses:
                    # Update the status of the cell
                    s = current_cell.get_spreading_prob()

                    if s > self.spread_threshold:
                        new_lecture_hall[i][j].set_status(Status.GOSSIP_SPREADER)

                    else:
                        new_lecture_hall[i][j].set_status(Status.SECRET_KEEPER)

        self.lecture_hall = new_lecture_hall

    def show_grid(self):
        grid = [[cell.get_status().value for cell in row] for row in self.lecture_hall]
        plt.imshow(grid)
        plt.colorbar()  # TODO: remove and add a proper label, this is mainly for testing purposes
        plt.show()

In [ ]:
size = 20
d = 0.7
spread_threshold = 0.6

g = Grid(size, d, spread_threshold)

In [ ]:
g.initialize_board()
g.set_spreader(13, 7)
g.show_grid()

In [ ]:
g.update_grid()
g.show_grid()